In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

import itertools

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("BatterySimV3.py"))
parent = os.path.dirname(os.path.dirname(current))
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import loadSeries, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel
from Logic import logic_bat, logic_DP
from Funcs_Logic_DP import logic_rollout, get_price, print_price_summary, logic_series_print, policy_rollout, DP_stochastic

In [2]:
house = "h16"
merged = merge(house)
merged

prod_h16  cons_h16  power_yield  SpotPriceDKK
2020-12-22 00:00:00       0.0       6.3         -6.3     11.090000
2020-12-22 01:00:00       0.0       4.8         -4.8     -0.150000
2020-12-22 02:00:00       0.0       4.8         -4.8    -55.880001
2020-12-22 03:00:00       0.0       4.3         -4.3    -72.919998
2020-12-22 04:00:00       0.0       4.0         -4.0    -53.720001
...                       ...       ...          ...           ...
2023-01-09 16:00:00       0.1       7.6         -7.5   1204.510010
2023-01-09 17:00:00       0.0      10.0        -10.0   1296.579956
2023-01-09 18:00:00       0.0       9.1         -9.1   1354.890015
2023-01-09 19:00:00       0.0      11.1        -11.1   1266.089966
2023-01-09 20:00:00       0.0       9.7         -9.7   1193.729980

[17975 rows x 4 columns]

## Running the logic models and DP

In [3]:
#"Input" variables
avg_yearly_cost_dict = {}
N = 7*24
bat_capacity=13
Start="2021-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N, freq="h")[-1]

In [4]:
# Define battery with capcacity 0, same as no battery
battery_no_bat = Battery(max_capacity=0,max_charge=0)

series_no_battery = logic_rollout(merged.loc[Start:End].copy(), battery_no_bat, logic_bat, get_price)

charge_list_no_bat = battery_no_bat.charge_list

avg_yearly_cost_dict["No battery"] = (print_price_summary(series_no_battery), 
                                      np.sum(charge_list_no_bat[charge_list_no_bat>0]), 
                                      np.sum(charge_list_no_bat[charge_list_no_bat<=0]))

Cost for period: 2021-06-19 00:00:00 to 2021-06-25 23:00:00 is:  322.0  DKK
Average cost per year is: 16889.0 DKK
Number of kwh purchased in the period: 539.3999999999942
Average number of kwh purchased per year: 28313.65508982005
Average number of kwh sold per year: 20445.251497005986


In [5]:
logic_series_print(series_no_battery)

hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   0.5962, -6.0000, -6.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.5770,  3.5770
    1:   0.5520, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0359,  6.6129
    2:   0.5306, -4.6000, -4.6000,  0.0000,  0.0000,  0.0000,  0.0000,  2.4406,  9.0535
    3:   0.5210, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  2.1884, 11.2419
    4:   0.5005, -3.6000, -3.6000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8017, 13.0436
    5:   0.4827, -3.9000, -3.9000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8825, 14.9260
    6:   0.4911, -2.9000, -2.9000,  0.0000,  0.0000,  0.0000,  0.0000,  1.4242, 16.3502
    7:   0.5466, -1.8000, -1.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.9838, 17.3340
    8:   0.5530, -1.2000, -1.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.6636, 17.9976
    9:   0.4828,  2.5000,  2.5000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1207, 17.8769
   10:   0.4341,  5.5000,  5.5

In [6]:
# Define battery with capcacity
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(merged.loc[Start:End].copy(), battery, logic_bat, get_price)

charge_list_bat = battery.charge_list

avg_yearly_cost_dict["Simple logic"] = (print_price_summary(series_battery), 
                                        np.sum(charge_list_bat[charge_list_bat>0]), 
                                        np.sum(charge_list_bat[charge_list_bat<=0]))

Cost for period: 2021-06-19 00:00:00 to 2021-06-25 23:00:00 is:  275.0  DKK
Average cost per year is: 14451.0 DKK
Number of kwh purchased in the period: 467.3999999999942
Average number of kwh purchased per year: 24534.30179640688
Average number of kwh sold per year: 14860.20718562874


In [7]:
logic_series_print(series_battery)

hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   0.5962, -6.0000, -6.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.5770,  3.5770
    1:   0.5520, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0359,  6.6129
    2:   0.5306, -4.6000, -4.6000,  0.0000,  0.0000,  0.0000,  0.0000,  2.4406,  9.0535
    3:   0.5210, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  2.1884, 11.2419
    4:   0.5005, -3.6000, -3.6000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8017, 13.0436
    5:   0.4827, -3.9000, -3.9000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8825, 14.9260
    6:   0.4911, -2.9000, -2.9000,  0.0000,  0.0000,  0.0000,  0.0000,  1.4242, 16.3502
    7:   0.5466, -1.8000, -1.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.9838, 17.3340
    8:   0.5530, -1.2000, -1.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.6636, 17.9976
    9:   0.4828,  2.5000,  0.0000,  0.0000,  2.5000,  0.0000,  2.5000, -0.0000, 17.9976
   10:   0.4341,  5.5000,  0.0

In [8]:
DPbat = Battery(max_capacity=bat_capacity)

DP = DPModel(N, Start, End, house, merged, DPbat)

J1, pi = DP_stochastic(DP)

167 

C:\Users\alexa\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)


0   

In [9]:
J, traj, acts = policy_rollout(DP,pi=lambda x, k: pi[k][x],x0=0.0)

np_acts = np.array(acts)
actions = pd.DataFrame(acts,columns=['charge','buy'])
actions.index = merged.loc[Start:End].index
J

266.1110682472115

In [10]:
# Define battery with capcacity 0, same as no battery
DPbat = Battery(max_capacity=bat_capacity,actions=actions)

series_DP_battery = logic_rollout(merged.loc[Start:End].copy(), DPbat, logic_DP, get_price)

charge_list_DP_bat = DPbat.charge_list

avg_yearly_cost_dict["DP"] = (print_price_summary(series_DP_battery), 
                                      np.sum(charge_list_DP_bat[charge_list_DP_bat>0]), 
                                      np.sum(charge_list_DP_bat[charge_list_DP_bat<=0]))

Cost for period: 2021-06-19 00:00:00 to 2021-06-25 23:00:00 is:  266.0  DKK
Average cost per year is: 13968.0 DKK
Number of kwh purchased in the period: 478.3000000001251
Average number of kwh purchased per year: 25106.453892222136
Average number of kwh sold per year: 15400.864670665555


In [11]:
logic_series_print(series_DP_battery)

hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   0.5962, -6.0000, -6.0000,  0.0000, -0.0000,  0.0000,  0.0000,  3.5770,  3.5770
    1:   0.5520, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  3.0359,  6.6129
    2:   0.5306, -4.6000, -4.6000,  0.0000, -0.0000,  0.0000,  0.0000,  2.4406,  9.0535
    3:   0.5210, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  2.1884, 11.2419
    4:   0.5005, -3.6000, -3.6000,  0.0000, -0.0000,  0.0000,  0.0000,  1.8017, 13.0436
    5:   0.4827, -3.9000, -3.9000,  0.0000, -0.0000,  0.0000,  0.0000,  1.8825, 14.9260
    6:   0.4911, -2.9000, -4.8000,  1.9000,  1.9000,  0.0000,  1.9000,  2.3572, 17.2833
    7:   0.5466, -1.8000, -0.0000,  0.0000, -1.8000,  1.8000,  0.0000,  0.0000, 17.2833
    8:   0.5530, -1.2000, -1.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.6636, 17.9469
    9:   0.4828,  2.5000,  2.5000,  0.0000, -0.0000,  0.0000,  0.0000, -0.1207, 17.8262
   10:   0.4341,  5.5000,  5.5